In [ ]:
# import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "Tx Generator"

In [ ]:
from case_code.code_downloader import CodeDownloader, get_metadata
from case_code.code_loader import CodeLoader
from case_code.code_transformer import transform

# Download all case code
downloader = CodeDownloader()
downloaded_count = downloader.download()

In [ ]:
metadata = get_metadata()
print(f"{'Total files downloaded:':<24} {metadata['total_downloads']}")
print(f"{'Total cases:':<24} {metadata['total_cases']}")

In [ ]:
import pandas as pd

# Transform the downloaded code to case
loader = CodeLoader()
transformed_data = await transform(loader=loader)

data = [(key.value, value) for key, value in transformed_data.items()]
df = pd.DataFrame(data, columns=["Title", "Count"])
print(df.to_string(index=False))

In [ ]:
from case_code.code_transformer import get_transformation_result

df = get_transformation_result()
filtered_df = df.loc[df["transformable"] == False]
filtered_df

In [ ]:
from case import get_retriever

retriever = get_retriever()

In [ ]:
retriever.invoke("Stake ETH with Lido and deposit to Eigenpie")

In [ ]:
retriever.invoke("Swap 10 USDC to USDT")

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from models import Case
from utils.model_selector import get_chat_model


template = """Generate a structure output based only on the following context:
{context}
Question: {question}"""
prompt = ChatPromptTemplate.from_template(template)

retriever = get_retriever()

model = get_chat_model()


def format_docs(docs):
    return "\n\n".join(str(doc.metadata) for doc in docs)


# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model.with_structured_output(Case)
)

In [ ]:
question = "Stake 0.03 ETH with Lido and deposit to Eigenpie"
result = rag_chain.invoke(question)
print(result.json())

In [ ]:
question = "Stake 0.03 ETH with Lido and deposit to Eigenpie"
result = rag_chain.invoke(question)
print(result.json())

In [ ]:
question = "Swap 100 USDT for USDC on Uniswap"
result = rag_chain.invoke(question)
print(result.json())

In [ ]:
question = "Swap 0.001 ETH for USDC on Uniswap"
result = rag_chain.invoke(question)
print(result.json())

In [ ]:
question = "Swap 20 USDC to USDT"
result = rag_chain.invoke(question)
print(result.json())

In [ ]:
question = """Yield Farming with Multiple Protocols
	Step 1: Deposit ETH into a AAVE to earn interest.
	Step 2: Use aToken (aETH) as collateral to borrow DAI.
	Step 3: Deposit the borrowed DAI into a Yearn Finance.
	Step 4: Use the Yearn vault tokens in a liquidity pool on Uniswap to earn trading fees."""
result = rag_chain.invoke(question)
print(result.json())